In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely import wkt

# def load_clean_demo(path):
#     raw = pd.read_csv(path, header=None)
#     raw.columns = raw.iloc[1]
#     df = raw.iloc[2:].copy()
#     df = df.rename(columns={'Geography': 'GEOID'})
#     return df


# income_df = load_clean_demo("NYC_ACSST5Y2020.S1901-Data.csv")
# education_df = load_clean_demo("NYC_ACSST5Y2020.S1501-Data.csv")
# transport_df = load_clean_demo("NYC_ACSDT5Y2020.B08201-Data.csv")
# population_df = load_clean_demo("NYC_DECENNIALDP2020.DP1-Data.csv")
demo_df = pd.read_csv("nyc_data_numeric.csv").drop('Unnamed: 0', axis=1).rename(columns={'Geography': 'GEOID'})

grid_df = pd.read_csv("grid_data_updated.csv")

tract_df = pd.read_csv("ny_census_tracts_with_data.csv")
tract_df['geometry'] = tract_df['geometry'].apply(wkt.loads)
tracts_gdf = gpd.GeoDataFrame(tract_df, geometry='geometry', crs="EPSG:4326")

def make_grid_poly(row):
    return box(row['Min Long'], row['Min Lat'], row['Max Long'], row['Max Lat'])

grid_df['geometry'] = grid_df.apply(make_grid_poly, axis=1)
grid_gdf = gpd.GeoDataFrame(grid_df, geometry='geometry', crs="EPSG:4326")


In [2]:
demo_df

,GEOID,Geography_name,total_population,population_density,male_percent,percent_inhouseholds,percent_marriedhouseholds,percent_cohabitingcouple,percent_solomale,percent_solofemale,...,percent_novehicles,percent_1vehicle_perperson,percent_lessthan1vehicle_perperson,under25y_percent_highschool,under25y_percent_somecollege,under25y_percent_bachelors,over25y_percent_highschool,over25y_percent_somecollege,over25y_percent_bachelors,over25y_percent_graduatedegree
0,1400000US36005000200,Census Tract 2; Bronx County; New York,4779,27624.277457,0.476878,0.998326,0.112994,0.022808,0.033271,0.030550,...,0.062984,0.089349,0.185394,0.427,0.308,0.000,0.262,0.196,0.150,0.194
1,1400000US36005000400,Census Tract 4; Bronx County; New York,6272,20976.588629,0.473374,0.997768,0.129624,0.024872,0.030134,0.035714,...,0.070153,0.078444,0.253508,0.375,0.340,0.179,0.220,0.327,0.161,0.128
2,1400000US36005001600,Census Tract 16; Bronx County; New York,5795,30989.304813,0.433650,0.960311,0.090250,0.019845,0.033477,0.071268,...,0.222088,0.026402,0.337532,0.394,0.290,0.083,0.288,0.298,0.131,0.035
3,1400000US36005001901,Census Tract 19.01; Bronx County; New York,2292,29012.658228,0.503927,0.985166,0.078970,0.055410,0.105148,0.061518,...,0.207243,0.052792,0.289267,0.386,0.241,0.138,0.199,0.296,0.218,0.126
4,1400000US36005001902,Census Tract 19.02; Bronx County; New York,1613,8911.602210,0.537508,0.647861,0.055797,0.021699,0.092374,0.050217,...,0.127092,0.021699,0.174210,0.110,0.495,0.176,0.322,0.157,0.143,0.028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2062,1400000US36081157901,Census Tract 1579.01; Queens County; New York,5184,14811.428571,0.484568,1.000000,0.196181,0.006944,0.026427,0.052855,...,0.025463,0.116319,0.192708,0.149,0.621,0.231,0.217,0.203,0.280,0.191
2063,1400000US36081157902,Census Tract 1579.02; Queens County; New York,3757,13465.949821,0.491616,0.984296,0.194038,0.005856,0.015172,0.021826,...,0.018632,0.081980,0.223050,0.129,0.261,0.578,0.237,0.199,0.230,0.182
2064,1400000US36081157903,Census Tract 1579.03; Queens County; New York,4489,17535.156250,0.491423,0.996881,0.191802,0.010693,0.021163,0.028514,...,0.008242,0.079305,0.162397,0.334,0.466,0.199,0.193,0.167,0.301,0.134
2065,1400000US36081161700,Census Tract 1617; Queens County; New York,4541,16756.457565,0.498569,0.995816,0.199516,0.009689,0.024664,0.027527,...,0.023123,0.091610,0.220216,0.190,0.520,0.221,0.207,0.290,0.224,0.214


In [3]:
intersections = gpd.overlay(tracts_gdf, grid_gdf, how='intersection')

intersections['intersection_area'] = intersections.geometry.area
tracts_gdf['tract_area'] = tracts_gdf.geometry.area

intersections = intersections.merge(
    tracts_gdf[['GEOID', 'tract_area']],
    on='GEOID'
)
intersections['area_ratio'] = intersections['intersection_area'] / intersections['tract_area']


/var/folders/sh/swzqgpqd64j6vh8lqzql0vfm0000gn/T/ipykernel_30558/2200848939.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersections['intersection_area'] = intersections.geometry.area
/var/folders/sh/swzqgpqd64j6vh8lqzql0vfm0000gn/T/ipykernel_30558/2200848939.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts_gdf['tract_area'] = tracts_gdf.geometry.area


In [4]:
demo_df = demo_df.drop(columns=['Geography_name']).set_index('GEOID').add_prefix(f"{"demographics"}_")

# demo_merged = (
#     merge_on_geo(income_df, 'income')
#     .join(merge_on_geo(education_df, 'edu'), how='outer')
#     .join(merge_on_geo(transport_df, 'trans'), how='outer')
#     .join(merge_on_geo(population_df, 'pop'), how='outer')
# )

intersections = intersections.set_index('GEOID').join(demo_df, how='left').reset_index()


In [5]:
for col in intersections.columns:
    if col not in ['GEOID', 'Grid ID', 'intersection_area', 'tract_area', 'area_ratio', 'geometry']:
        intersections[col] = pd.to_numeric(intersections[col], errors='coerce')

feature_cols = [
    col for col in intersections.columns
    if col not in ['GEOID', 'Grid ID', 'intersection_area', 'tract_area', 'area_ratio', 'geometry']
]

for col in feature_cols:
    intersections[col] = intersections[col] * intersections['area_ratio']

cleaned_data = intersections.groupby('Grid ID')[feature_cols].sum().reset_index()

In [6]:
cleaned_data = cleaned_data.drop(['GEO_ID', 'Unnamed: 0'], axis=1, errors='ignore')
cleaned_data = cleaned_data.drop(['Min Lat', 'Max Lat', 'Min Long', 'Max Long', 'Grid ID'], axis=1).dropna().reset_index().drop('index', axis=1)
cleaned_data = cleaned_data[cleaned_data['Ride Count'] != 0]
cleaned_data = cleaned_data.dropna()
cleaned_data.to_csv("nyc_grid_demographics.csv", index=False)

In [7]:
cleaned_data

,NAMELSAD,AREALAND,AREAWATR,AREALAND_SQMI,AREAWATR_SQMI,Ride Count,demographics_total_population,demographics_population_density,demographics_male_percent,demographics_percent_inhouseholds,...,demographics_percent_novehicles,demographics_percent_1vehicle_perperson,demographics_percent_lessthan1vehicle_perperson,demographics_under25y_percent_highschool,demographics_under25y_percent_somecollege,demographics_under25y_percent_bachelors,demographics_over25y_percent_highschool,demographics_over25y_percent_somecollege,demographics_over25y_percent_bachelors,demographics_over25y_percent_graduatedegree
569,0.0,60104.409463,32536.554849,0.023217,0.012538,809.935123,76.145174,711.637137,0.031884,0.061408,...,0.011956,0.001416,0.016414,0.028057,0.009163,0.004866,0.015039,0.008531,0.007772,0.008215
570,0.0,79995.754737,12649.840329,0.030901,0.004875,3038.773156,330.459432,3191.985274,0.117974,0.225244,...,0.045286,0.005247,0.062432,0.100633,0.041183,0.016746,0.056968,0.030053,0.029714,0.029208
690,0.0,89244.065018,3392.938834,0.034439,0.001313,19772.617405,1345.527712,18864.765244,0.138909,0.290828,...,0.063316,0.009432,0.127369,0.069519,0.023165,0.192250,0.017930,0.040644,0.091003,0.121762
745,0.0,92633.143638,0.000000,0.035904,0.000000,20742.064547,2085.425839,51257.291157,0.420895,0.886151,...,0.159645,0.031411,0.341106,0.118462,0.103425,0.582423,0.055746,0.076012,0.352063,0.362827
747,0.0,85376.275191,7260.663432,0.032947,0.002811,2555.760654,323.627033,4978.877431,0.032849,0.067889,...,0.016682,0.002234,0.030185,0.011777,0.000000,0.056298,0.002859,0.011709,0.018789,0.031859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3966,0.0,92357.255848,0.000000,0.035718,0.000000,4.777191,4348.531954,130143.383272,0.451416,0.949253,...,0.248525,0.008099,0.309856,0.181861,0.379098,0.035780,0.244089,0.244168,0.089245,0.021227
3968,0.0,92359.250116,0.000000,0.035781,0.000000,0.823235,2757.698171,43487.847485,0.193487,0.343330,...,0.090033,0.006870,0.112034,0.069058,0.217857,0.054901,0.124704,0.108196,0.039548,0.028061
3969,0.0,91625.954210,739.959426,0.035377,0.000280,0.542754,355.875310,2283.541423,0.032485,0.029606,...,0.009052,0.000562,0.010509,0.018166,0.048322,0.002561,0.024578,0.013284,0.003835,0.015061
3992,0.0,92355.172634,0.000000,0.035815,0.000000,2.656497,4320.687619,117522.663147,0.412996,0.870088,...,0.215215,0.010614,0.272962,0.343343,0.362982,0.059897,0.303213,0.147286,0.129104,0.042206
